In [1]:
import iias
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from keras.models import Model
from vit_keras import vit, utils
from numpy.linalg import norm
from sklearn.decomposition import PCA
from glob import glob

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/home/abhishekmandal/anaconda3/envs/TensorFlow/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### CNNs

### VGG16

In [2]:
from tensorflow.keras.applications.vgg16 import preprocess_input

vgg16_biased = keras.models.load_model('selected_models/cnn/biased/VGG16biased_3.h5')
vgg16_biased = tf.keras.models.Sequential(vgg16_biased.layers[:-4])
vgg16_unbiased = keras.models.load_model('selected_models/cnn/unbiased/VGG16unbiased_3.h5')
vgg16_unbiased = tf.keras.models.Sequential(vgg16_unbiased.layers[:-4])

2023-04-17 16:04:35.910349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 16:04:35.915454: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 16:04:35.915632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 16:04:35.916122: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
vgg_res = iias.cnn_iias_calculator(vgg16_biased, vgg16_unbiased, preprocess_input, 'vgg_16' )
vgg_res

2023-04-17 16:04:37.157225: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-17 16:04:37.803090: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2023-04-17 16:04:38.648807: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-17 16:04:38.649491: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-17 16:04:38.649506: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2023-04-17 16:04:38.650229: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-17 16:04:38.650274: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx co

[[-0.0880601, -0.06210778, -0.1349569, -0.09184347],
 [-0.032288957, -0.027997797, -0.08637945, -0.06640013]]

### Inception v3

In [4]:
from tensorflow.keras.applications.inception_v3 import  preprocess_input

inception_biased = keras.models.load_model('selected_models/cnn/biased/Inceptionbiased_2.h5')
inception_biased = Model(inception_biased.input, inception_biased.layers[-5].output)
inception_unbiased = keras.models.load_model('selected_models/cnn/unbiased/Inceptionunbiased_2.h5')
inception_unbiased = Model(inception_unbiased.input, inception_unbiased.layers[-5].output)

inception_res = iias.cnn_iias_calculator(inception_biased, inception_unbiased, preprocess_input, 'inception')
inception_res

[[0.04968613, 0.011103727, 0.011259139, 0.039424554],
 [-0.0070485966, -0.028550252, -0.018676803, -0.0189191]]

### ResNet 152

In [5]:
from tensorflow.keras.applications.resnet_v2 import  preprocess_input

resnet_biased = keras.models.load_model('selected_models/cnn/biased/ResNet152biased_4.h5')
resnet_biased = Model(resnet_biased.input, resnet_biased.layers[-15].output)
resnet_unbiased = keras.models.load_model('selected_models/cnn/unbiased/ResNet152unbiased_4.h5')
resnet_unbiased = Model(resnet_unbiased.input, resnet_unbiased.layers[-15].output)

resnet_res = iias.cnn_iias_calculator(resnet_biased, resnet_unbiased, preprocess_input, 'resnet')
resnet_res

[[0.034034707, -0.02623316, 0.030634433, 0.03591928],
 [0.04883014, -0.046174467, 0.03265802, 0.055762947]]

### Xception

In [6]:
from tensorflow.keras.applications.xception import  preprocess_input

xception_biased = keras.models.load_model('selected_models/cnn/biased/Xceptionbiased_1.h5')
xception_biased = Model(xception_biased.input, xception_biased.layers[-5].output)
xception_unbiased = keras.models.load_model('selected_models/cnn/unbiased/Xceptionunbiased_1.h5')
xception_unbiased = Model(xception_unbiased.input, xception_unbiased.layers[-5].output)

xception_res = iias.cnn_iias_calculator(xception_biased, xception_unbiased, preprocess_input, 'xception')
xception_res

[[-0.016228886, 0.0046228347, 0.008664374, -0.0012554636],
 [0.019838614, 0.08366705, 0.04603773, 0.04557107]]

### Vision Transformer

### ViT - B/16

In [7]:
vitb16_biased = keras.models.load_model('selected_models/vit/biased/vit_b16biased_3.h5')
vitb16_biased = tf.keras.models.Sequential(vitb16_biased.layers[:-5])
vitb16_unbiased = keras.models.load_model('selected_models/vit/unbiased/vit_b16unbiased_3.h5')
vitb16_unbiased = tf.keras.models.Sequential(vitb16_unbiased.layers[:-5])

vitb16 = iias.vit_iias_calculator(vitb16_biased, vitb16_unbiased, 112)
vitb16

/home/abhishekmandal/anaconda3/envs/TensorFlow/lib/python3.9/site-packages/tensorflow_addons/activations/gelu.py:79: UserWarning: Default value of `approximate` is changed from `True` to `False`
  warnings.warn(
2023-04-17 16:10:27.313850: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[[-0.0012160337, 6.4396672e-06, -0.00027921793, 0.00026151718],
 [-0.036848247, -0.0668989, -0.07934042, -0.063547686]]

### ViT - B/32

In [8]:
vitb32_biased = keras.models.load_model('selected_models/vit/biased/vit_b32biased_1.h5')
vitb32_biased = tf.keras.models.Sequential(vitb32_biased.layers[:-5])
vitb32_unbiased = keras.models.load_model('selected_models/vit/unbiased/vit_b32unbiased_1.h5')
vitb32_unbiased = tf.keras.models.Sequential(vitb32_unbiased.layers[:-5])

vitb32 = iias.vit_iias_calculator(vitb32_biased, vitb32_unbiased, 224)
vitb32

[[-0.0024462878, -0.0020258767, -0.0021286956, -0.0018057056],
 [-0.009264702, -0.007058598, -0.010999792, -0.0013921097]]

### ViT - L/16

In [9]:
vitl16_biased = keras.models.load_model('selected_models/vit/biased/vit_l16biased_4.h5')
vitl16_biased = tf.keras.models.Sequential(vitl16_biased.layers[:-5])
vitl16_unbiased = keras.models.load_model('selected_models/vit/unbiased/vit_l16unbiased_4.h5')
vitl16_unbiased = tf.keras.models.Sequential(vitl16_unbiased.layers[:-5])

vitl16 = iias.vit_iias_calculator(vitl16_biased, vitl16_unbiased, 112)
vitl16

[[0.057774507, 0.0836375, 0.094554, 0.11902785],
 [0.009964774, 0.006115029, 0.010184256, 0.011635155]]

### ViT - L/32

In [10]:
vitl32_biased = keras.models.load_model('selected_models/vit/biased/vit_l32biased_5.h5')
vitl32_biased = tf.keras.models.Sequential(vitl32_biased.layers[:-5])
vitl32_unbiased = keras.models.load_model('selected_models/vit/unbiased/vit_l32unbiased_5.h5')
vitl32_unbiased = tf.keras.models.Sequential(vitl32_unbiased.layers[:-5])

vitl32 = iias.vit_iias_calculator(vitl32_biased, vitl32_unbiased, 224)
vitl32

[[-0.011262396, -0.005719486, -0.0064244913, -0.006038543],
 [0.009800295, -0.01767917, 0.009077253, 6.4959095e-05]]

In [11]:
target_ls = ['ceo', 'engineer', 'nurse', 'school_teacher']

df = pd.DataFrame(list(zip(target_ls, vgg_res[0], inception_res[0], resnet_res[0], xception_res[0], vitb16[0], vitb32[0], vitl16[0], vitl32[0], 
                           vgg_res[1], inception_res[1], resnet_res[1], xception_res[1], vitb16[1], vitb32[1], vitl16[1], vitl32[1])),
                 columns=['Target', 'VGG16_biased', 'Inceptionv3_biased', 'ResNet152_biased','Xception_biased', 'ViT-B_16_biased', 'ViT-B_32_biased', 'ViT-L_16_biased', 'ViT-L_32_biased',
                         'VGG16_unbiased', 'Inceptionv3_unbiased', 'ResNet152_unbiased','Xception_unbiased', 'ViT-B_16_unbiased', 'ViT-B_32_unbiased', 'ViT-L_16_unbiased', 'ViT-L_32_unbiased'])

In [12]:
df.to_csv('results/iias_obscured.csv', index=False)

### Result Analysis

In [13]:
print('Model ',target_ls)
print('VGG16: ', [x + y for x, y in zip(vgg_res[0], vgg_res[1])])
print('Inceptionv3: ', [x + y for x, y in zip(inception_res[0], inception_res[1])])
print('ResNet152: ', [x + y for x, y in zip(resnet_res[0], resnet_res[1])])
print('Xception: ', [x + y for x, y in zip(xception_res[0], xception_res[1])])
print('ViT B/16: ', [x + y for x, y in zip(vitb16[0], vitb16[1])])
print('ViT B/32: ', [x + y for x, y in zip(vitb32[0], vitb32[1])])
print('ViT L/16: ', [x + y for x, y in zip(vitl16[0], vitl16[1])])
print('ViT L/32: ', [x + y for x, y in zip(vitl32[0], vitl32[1])])

Model  ['ceo', 'engineer', 'nurse', 'school_teacher']
VGG16:  [-0.120349064, -0.09010558, -0.22133635, -0.1582436]
Inceptionv3:  [0.042637534, -0.017446525, -0.007417664, 0.020505454]
ResNet152:  [0.08286485, -0.072407626, 0.06329245, 0.091682225]
Xception:  [0.003609728, 0.08828988, 0.054702103, 0.044315606]
ViT B/16:  [-0.038064282, -0.06689246, -0.07961964, -0.06328617]
ViT B/32:  [-0.01171099, -0.009084474, -0.013128487, -0.0031978153]
ViT L/16:  [0.06773928, 0.089752525, 0.10473826, 0.13066301]
ViT L/32:  [-0.001462101, -0.023398656, 0.0026527615, -0.0059735836]


In [14]:
print('Model ',target_ls)
print('VGG16: ', [abs(x) + abs(y) for x, y in zip(vgg_res[0], vgg_res[1])])
print('Inceptionv3: ', [abs(x) + abs(y) for x, y in zip(inception_res[0], inception_res[1])])
print('ResNet152: ', [abs(x) + abs(y) for x, y in zip(resnet_res[0], resnet_res[1])])
print('Xception: ', [abs(x) + abs(y) for x, y in zip(xception_res[0], xception_res[1])])
print('ViT B/16: ', [abs(x) + abs(y) for x, y in zip(vitb16[0], vitb16[1])])
print('ViT B/32: ', [abs(x) + abs(y) for x, y in zip(vitb32[0], vitb32[1])])
print('ViT L/16: ', [abs(x) + abs(y) for x, y in zip(vitl16[0], vitl16[1])])
print('ViT L/32: ', [abs(x) + abs(y) for x, y in zip(vitl32[0], vitl32[1])])

Model  ['ceo', 'engineer', 'nurse', 'school_teacher']
VGG16:  [0.120349064, 0.09010558, 0.22133635, 0.1582436]
Inceptionv3:  [0.056734726, 0.03965398, 0.029935941, 0.058343653]
ResNet152:  [0.08286485, 0.072407626, 0.06329245, 0.091682225]
Xception:  [0.0360675, 0.08828988, 0.054702103, 0.046826534]
ViT B/16:  [0.038064282, 0.066905335, 0.07961964, 0.0638092]
ViT B/32:  [0.01171099, 0.009084474, 0.013128487, 0.0031978153]
ViT L/16:  [0.06773928, 0.089752525, 0.10473826, 0.13066301]
ViT L/32:  [0.02106269, 0.023398656, 0.015501744, 0.006103502]


In [15]:
print('Model ',target_ls)
print('VGG16: ', sum([abs(x) + abs(y) for x, y in zip(vgg_res[0], vgg_res[1])]))
print('Inceptionv3: ', sum([abs(x) + abs(y) for x, y in zip(inception_res[0], inception_res[1])]))
print('ResNet152: ', sum([abs(x) + abs(y) for x, y in zip(resnet_res[0], resnet_res[1])]))
print('Xception: ', sum([abs(x) + abs(y) for x, y in zip(xception_res[0], xception_res[1])]))
print('ViT B/16: ', sum([abs(x) + abs(y) for x, y in zip(vitb16[0], vitb16[1])]))
print('ViT B/32: ', sum([abs(x) + abs(y) for x, y in zip(vitb32[0], vitb32[1])]))
print('ViT L/16: ', sum([abs(x) + abs(y) for x, y in zip(vitl16[0], vitl16[1])]))
print('ViT L/32: ', sum([abs(x) + abs(y) for x, y in zip(vitl32[0], vitl32[1])]))

Model  ['ceo', 'engineer', 'nurse', 'school_teacher']
VGG16:  0.5900345891714096
Inceptionv3:  0.18466829881072044
ResNet152:  0.31024715304374695
Xception:  0.2258860170841217
ViT B/16:  0.24839845672249794
ViT B/32:  0.037121767178177834
ViT L/16:  0.39289306849241257
ViT L/32:  0.06606659339740872


In [16]:
print('Model ',target_ls)
print('VGG16: ', abs(sum([x + y for x, y in zip(vgg_res[0], vgg_res[1])])))
print('Inceptionv3: ', abs(sum([x + y for x, y in zip(inception_res[0], inception_res[1])])))
print('ResNet152: ', abs(sum([x + y for x, y in zip(resnet_res[0], resnet_res[1])])))
print('Xception: ', abs(sum([x + y for x, y in zip(xception_res[0], xception_res[1])])))
print('ViT B/16: ', abs(sum([x + y for x, y in zip(vitb16[0], vitb16[1])])))
print('ViT B/32: ', abs(sum([x + y for x, y in zip(vitb32[0], vitb32[1])])))
print('ViT L/16: ', abs(sum([x + y for x, y in zip(vitl16[0], vitl16[1])])))
print('ViT L/32: ', abs(sum([x + y for x, y in zip(vitl32[0], vitl32[1])])))

Model  ['ceo', 'engineer', 'nurse', 'school_teacher']
VGG16:  0.5900345891714096
Inceptionv3:  0.03827879950404167
ResNet152:  0.1654319018125534
Xception:  0.19091731682419777
ViT B/16:  0.2478625513613224
ViT B/32:  0.037121767178177834
ViT L/16:  0.39289306849241257
ViT L/32:  0.028181579429656267


In [17]:
model_ls = ['VGG16', 'Inceptionv3', 'ResNet152', 'Xception', 'ViT B/16', 'ViT B/32', 'ViT L/16', 'ViT L/32']
result_ls = [vgg_res, inception_res, resnet_res, xception_res, vitb16, vitb32, vitl16, vitl32]

print('Model     IIAS')
cnn_iias = []
vit_iias = []
for i, result in enumerate(result_ls):
    bias = abs(sum([x + y for x, y in zip(result[0], result[1])]))
    print(model_ls[i]+': ', bias)
    if i<4 :
        cnn_iias.append(bias)
    else:
        vit_iias.append(bias)

print('Mean CNN IIAS: ', np.mean(cnn_iias))
print('Mean ViT IIAS: ', np.mean(vit_iias))

Model     IIAS
VGG16:  0.5900345891714096
Inceptionv3:  0.03827879950404167
ResNet152:  0.1654319018125534
Xception:  0.19091731682419777
ViT B/16:  0.2478625513613224
ViT B/32:  0.037121767178177834
ViT L/16:  0.39289306849241257
ViT L/32:  0.028181579429656267
Mean CNN IIAS:  0.2461656518280506
Mean ViT IIAS:  0.17651474161539227


In [18]:
print('Model     IIAS')
cnn_iias = []
vit_iias = []
for i, result in enumerate(result_ls):
    bias = sum([abs(x) + abs(y) for x, y in zip(result[0], result[1])])
    print(model_ls[i]+': ', bias)
    if i<4 :
        cnn_iias.append(bias)
    else:
        vit_iias.append(bias)

print('Mean CNN IIAS: ', np.mean(cnn_iias))
print('Mean ViT IIAS: ', np.mean(vit_iias))

Model     IIAS
VGG16:  0.5900345891714096
Inceptionv3:  0.18466829881072044
ResNet152:  0.31024715304374695
Xception:  0.2258860170841217
ViT B/16:  0.24839845672249794
ViT B/32:  0.037121767178177834
ViT L/16:  0.39289306849241257
ViT L/32:  0.06606659339740872
Mean CNN IIAS:  0.3277090145274997
Mean ViT IIAS:  0.18611997144762427


In [19]:
print('Model     IIAS')
cnn_iias = []
vit_iias = []
for i, result in enumerate(result_ls):
    bias = sum([abs(x + y) for x, y in zip(result[0], result[1])])
    print(model_ls[i]+': ', bias)
    if i<4 :
        cnn_iias.append(bias)
    else:
        vit_iias.append(bias)

print('Mean CNN IIAS: ', np.mean(cnn_iias))
print('Mean ViT IIAS: ', np.mean(vit_iias))

Model     IIAS
VGG16:  0.5900345891714096
Inceptionv3:  0.08800717815756798
ResNet152:  0.31024715304374695
Xception:  0.19091731682419777
ViT B/16:  0.2478625513613224
ViT B/32:  0.037121767178177834
ViT L/16:  0.39289306849241257
ViT L/32:  0.03348710248246789
Mean CNN IIAS:  0.2948015592992306
Mean ViT IIAS:  0.17784112237859517


In [20]:
for k, bias in enumerate(['biased', 'unbiased']):
    total_bias_cnn = []
    total_bias_vit = []
    print(bias)
    print('='*50)
    for j, target in enumerate(target_ls):

        print(target)

        cnn_iias = []
        vit_iias = []
        for i,result in enumerate(result_ls):
            if i<4 :
                cnn_iias.append(result[k][j])
            else:
                vit_iias.append(result[k][j])
        print('Mean CNN IIAS: ', abs(np.mean(cnn_iias)))
        print('Mean ViT IIAS: ', abs(np.mean(vit_iias)))
        total_bias_cnn.append(abs(np.mean(cnn_iias)))
        total_bias_vit.append(abs(np.mean(vit_iias)))
        print('\n')
        
    print('-'*50)
    print('Total IIAS CNN: ', np.mean(total_bias_cnn))
    print('Total IIAS ViT: ', np.mean(total_bias_vit))
    print('-'*50)
    print('\n')
        
    

biased
ceo
Mean CNN IIAS:  0.005142038
Mean ViT IIAS:  0.010712447


engineer
Mean CNN IIAS:  0.018153595
Mean ViT IIAS:  0.018974645


nurse
Mean CNN IIAS:  0.021099737
Mean ViT IIAS:  0.021430397


school_teacher
Mean CNN IIAS:  0.0044387756
Mean ViT IIAS:  0.027861282


--------------------------------------------------
Total IIAS CNN:  0.012208537
Total IIAS ViT:  0.019744692
--------------------------------------------------


unbiased
ceo
Mean CNN IIAS:  0.0073328004
Mean ViT IIAS:  0.006586969


engineer
Mean CNN IIAS:  0.0047638677
Mean ViT IIAS:  0.02138041


nurse
Mean CNN IIAS:  0.006590127
Mean ViT IIAS:  0.017769676


school_teacher
Mean CNN IIAS:  0.004003695
Mean ViT IIAS:  0.013309921


--------------------------------------------------
Total IIAS CNN:  0.0056726225
Total IIAS ViT:  0.014761744
--------------------------------------------------


